# Welcome to Stable Diffusion WebUI 1.4!

This colab runs the latest webui version from the repo https://github.com/hlky/stable-diffusion-webui

---

If this colab helped you, support me on ko-fi and don't forget to subscribe to my awesome list https://github.com/altryne/awesome-ai-art-image-synthesis


## 1 -  Setup stage

### 1.0

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cbc82f0b-9a6b-eeb7-bdba-fcdd873f74d4)


### 1.1 Download repo and install

Clone git repo and setup miniconda


In [ ]:
#@markdown ## Download the stable-diffusion webui repo
#@markdown And install colab related conda (uncheck stable version if you like new features)
%cd /content
!git clone https://github.com/hlky/stable-diffusion
%cd /content/stable-diffusion
stable_version = False #@param {type:"boolean"}
if stable_version: 
  !git checkout colab-pin

/content
Cloning into 'stable-diffusion'...
remote: Enumerating objects: 1159, done.
remote: Total 1159 (delta 0), reused 0 (delta 0), pack-reused 1159
Receiving objects: 100% (1159/1159), 42.83 MiB | 27.99 MiB/s, done.
Resolving deltas: 100% (567/567), done.
/content/stable-diffusion
--2022-09-06 08:32:49--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  73.06M  86.8MB/s    in 0.8s    

2022-09-06 08:32:50 (86.8 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [76607678/76607678]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment lo

In [ ]:
# import sys
# !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x Miniconda3-latest-Linux-x86_64.sh
# !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# sys.path.append('/usr/local/lib/python3.7/site-packages/')
# !rm Miniconda3-latest-Linux-x86_64.sh

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

### 1.2 Environment setup
Setup environment, Gfpgan and Real-ESRGAN. Takes about 5-6 minutes

In [ ]:
#@markdown ### Set up conda environment - Takes a while
!conda env update -n base -f /content/stable-diffusion/environment.yaml 

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

intel-openmp-2022.0. | 4.2 MB    | : 100% 1.0/1 [00:00<00:00,  4.04it/s]
bzip2-1.0.8          | 78 KB     | : 100% 1.0/1 [00:00<00:00, 17.95it/s]
typing_extensions-4. | 42 KB     | : 100% 1.0/1 [00:00<00:00, 17.92it/s]
gnutls-3.6.15        | 1.0 MB    | : 100% 1.0/1 [00:00<00:00, 12.32it/s]
conda-package-handli | 885 KB    | : 100% 1.0/1 [00:00<00:00, 13.55it/s]
mkl_fft-1.3.0        | 177 KB    | : 100% 1.0/1 [00:00<00:00, 14.60it/s]
jpeg-9e              | 240 KB    | : 100% 1.0/1 [00:00<00:00, 15.45it/s]
mkl_random-1.1.1     | 341 KB    | : 100% 1.0/1 [00:00<00:00, 12.79it/s]
lame-3.100           | 323 KB    | : 100% 1.0/1 [00:00<00:00, 15.44it/s]
nettle-3.7.3         | 809 KB    | : 100% 1.0/1 [00:00<00:00, 13.22i

### 1.3 Setup Upscalers - CFPGan and ESRGAN

In [ ]:

#@markdown ### Build upscalers support
#@markdown **GFPGAN** Automatically correct distorted faces with a built-in GFPGAN option, fixes them in less than half a second
#@markdown **ESRGAN** Boosts the resolution of images with a built-in RealESRGAN option
#@markdown LDSR and GoBig enable amazing upscale options in the new Image Lab

add_CFP = True #@param {type:"boolean"} 
add_ESR = True #@param {type:"boolean"}
add_LDSR = True #@param {type:"boolean"} 
#@markdown ⚠️ LDSR is 1.9GB and make take time to download

if add_CFP:
  %cd /content/stable-diffusion/src/gfpgan/
  !pip install basicsr facexlib yapf lmdb opencv-python pyyaml tb-nightly --no-deps
  !python setup.py develop
  !pip install realesrgan
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
if add_ESR:
  %cd /content/stable-diffusion/src/realesrgan/
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
if add_LDSR:
  %cd /content/stable-diffusion/src
  !git clone https://github.com/devilismyfriend/latent-diffusion
  %cd latent-diffusion
  %mkdir -p experiments/
  %cd experiments/
  %mkdir -p pretrained_models
  %cd pretrained_models
  #project.yaml download
  !wget -O project.yaml https://heibox.uni-heidelberg.de/f/31a76b13ea27482981b4/?dl=1
  #model.ckpt model download
  !wget -O model.ckpt https://heibox.uni-heidelberg.de/f/578df07c8fc04ffbadf3/?dl=1

  %cd /content/stable-diffusion/
  !wget https://github.com/matomo-org/travis-scripts/blob/master/fonts/Arial.ttf?raw=true -O arial.ttf

/content/stable-diffusion/src/gfpgan
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/site-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
  warnings.warn(
running develop
/usr/local/lib/python3.8/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running egg_info
writing gfpgan.egg-info/PKG-INFO
writing dependency_links to gfpgan.egg-info/dependency_links.txt
writing requirements to gfpgan.egg-info/requires.txt
writing top-level names to gfpgan.egg-info/top_level.txt

### 1.4 Connect to Google Drive

In [ ]:
#@markdown # Load the stable-diffusion model

#@markdown **Model Path Variables**
# ask for the link
print("Local Path Variables:\n")

models_path = "/content/models" #@param {type:"string"}
output_path = "/content/output" #@param {type:"string"}

%cd /content/

import os

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

#@markdown # Download model 1.4 from your Google Drive link

#@markdown Copy only your Google Drive link **key**

#@markdown Example : drive.google.com/file/d/**18n2HARwX_sZuEZbBzJsPDKudeUy_IFbH**/view?usp=sharing

#@markdown Your file should be named sd-v1-4.ckpt

KeyGDrivelink = "" #@param {type:"string"}

my_file_id = KeyGDrivelink

!gdown $my_file_id
!mv 'sd-v1-4.ckpt' '{models_path}/'

from IPython.display import clear_output 
clear_output()

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

models_path: /content/models
output_path: /content/output


## 2 - Run the Stable Diffusion webui

### 2.1 Optional - Set webUI settings and configs before running 

In [ ]:
#@markdown # Launch preferences - Advanced
share_password="" #@param {type:"string"}
#@markdown * Add a password to your webui
defaults="configs/webui/webui.yaml" #@param {type:"string"}
#@markdown * path to configuration file providing UI defaults, uses same format as cli parameter)  
#@markdown Edit this file if you want to change the default settings UI launches with

#@markdown ---
save_metadata = False #@param {type:"boolean"}
#@markdown * Whether to embed the generation parameters in the sample images
skip_grid = False #@param {type:"boolean"}
#@markdown * Do not save a grid, only individual samples. Helpful when evaluating lots of samples
skip_save = False #@param {type:"boolean"}
#@markdown * Do not save individual samples as files. For speed measurements
optimized = False #@param {type:"boolean"}
#@markdown * Load the model onto the device piecemeal instead of all at once to reduce VRAM usage at the cost of performance
optimized_turbo = True #@param {type:"boolean"}
#@markdown * Alternative optimization mode that does not save as much VRAM but runs siginificantly faster
no_verify_input = False #@param {type:"boolean"}
#@markdown * Do not verify input to check if it's too long
no_half = False #@param {type:"boolean"}
#@markdown * Do not switch the model to 16-bit floats
no_progressbar_hiding = True #@param {type:"boolean"}
#@markdown * Do not hide progressbar in gradio UI
extra_models_cpu = False #@param {type:"boolean"}
#@markdown * Run extra models (GFGPAN/ESRGAN) on cpu
esrgan_cpu = True #@param {type:"boolean"}
#@markdown * run ESRGAN on cpu
gfpgan_cpu = False #@param {type:"boolean"}
#@markdown * run GFPGAN on cpu


run_string_with_variables = {
 '--save-metadata': f'{save_metadata}',
 '--skip-grid': f'{skip_grid}',
 '--skip-save': f'{skip_save}',
 '--optimized': f'{optimized}',
 '--optimized-turbo': f'{optimized_turbo}',
 '--no-verify-input': f'{no_verify_input}',
 '--no-half': f'{no_half}',
 '--no-progressbar-hiding': f'{no_progressbar_hiding}',
 '--extra-models-cpu': f'{extra_models_cpu}',
 '--esrgan-cpu': f'{esrgan_cpu}',
 '--gfpgan-cpu': f'{gfpgan_cpu}'}

only_true_vars = {k for (k,v) in run_string_with_variables.items() if v == 'True'}
vars = " ".join(only_true_vars)


## 3 - Launch WebUI for stable diffusion

In [ ]:
%cd /content/stable-diffusion
!npm install -g localtunnel
!nohup lt -p 7860 > lt.log 2>&1 &
!sleep 3
!cat /content/stable-diffusion/lt.log

In [ ]:
#@markdown ** keep in mind that this script is set to run for ever, google will disconnect you after 90 minutes on free tiers

#@markdown # Important - click the public URL to launch WebUI in another tab
#@markdown ![](https://user-images.githubusercontent.com/463317/187105407-dd9b0f4e-c8da-49d3-8c78-1767f5c9aa83.jpg)

#fix adding share_password to the launch params, and also changin {vars} to $vars as it was causing webui.py to fail.


%cd /content/stable-diffusion

if share_password == "":
  !python scripts/webui.py \
  --ckpt '{models_path}/sd-v1-4.ckpt' \
  --outdir '{output_path}' \
  --share $vars
else:
  !python scripts/webui.py \
  --ckpt '{models_path}/sd-v1-4.ckpt' \
  --outdir '{output_path}' \
  --share-password '{share_password}' \
  --share $vars



/content/stable-diffusion
Found GFPGAN
Found RealESRGAN
Found LDSR
Loading model from /content/models/sd-v1-4.ckpt
Global Step: 470000
UNet: Running in eps-prediction mode
CondStage: Running in eps-prediction mode
FirstStage: Running in eps-prediction mode
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
/usr/local/lib/python3.8/site-packages/gradio/deprecation.py:43: UserWarning: You have unused kwarg parameters in File, please remove them: {'source': 'upload'}
  warnings.warn(
Running on local URL:  http://localhost:7860/
Running on public URL: https://23466.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
[MemMon] Recording max memory usage...

Iteration: 1/1
100% 50/50 [00:16<00:00,  3.12it/s]
[MemMon] Stopped recording.

[MemMon] Recording max memory usage...

Iteration: 1/1
 50% 25/50 [